In [2]:
import requests
import pandas as pd
import os
from pathlib import Path
import numpy as np

In [3]:
baseurl = "https://api.wikiwho.net/en/api/v1.0.0-beta/"
content = "Yugoslavia"
def _url(url, path):
    return url + path
revisions_url = _url( baseurl,"rev_ids/")
mama_revisions_url = _url(revisions_url, content+"/")
params = {"editor": "true", "timestamp": "true"}
response = requests.get(mama_revisions_url, params= params)
revisons = pd.DataFrame(response.json()["revisions"])

In [4]:
len(response.json()["revisions"])

3688

In [4]:
revisons["id"].describe()

count    2.536000e+03
mean     3.424353e+08
std      2.385816e+08
min      3.317807e+06
25%      1.365433e+08
50%      2.252330e+08
75%      5.517890e+08
max      8.412142e+08
Name: id, dtype: float64

In [5]:
filename = content + ".h5"
with pd.HDFStore( filename, 'r') as store1:
    df_3317807 = store1.get("/3317807")
    df_3317866 = store1.get("3317866")
    print(df_3317807.head(2))
    print("\n")   
    print(df_3317866.head(2))

   in  o_rev_id        out str  token_id  inserted  deleted
0  []   3317807  [3317921]  an         0      True    False
1  []   3317807  [3317921]  80         1      True    False


   in  o_rev_id        out str  token_id  inserted  deleted
0  []   3317807  [3317921]  an         0     False     True
1  []   3317807  [3317921]  80         1     False     True


In [189]:
filename = content + ".h5"
revisions = []
with pd.HDFStore( filename, 'r') as store:
    for revision in store.keys():
        revisions.append(store.get(revision))

### code to find epsillon neighbourhood

In [233]:
contexts = []
subjects = []
for revision in revisions:
    inserted_array = np.array(revision["inserted"])
    inserted_pos = np.argwhere(inserted_array == True)
    if inserted_pos.shape[0] == 0:
        continue
    total_tokens = np.array(revision["token_id"])
    total_tokens = total_tokens.reshape(total_tokens.shape[0],1)
    inserted_tokens = total_tokens[inserted_pos]
    epsilon = 5
    context =  []
# timing to select concatinate rather than vstack
# %timeit np.vstack((total_tokens[pos - epsilon: pos +epsilon], context))
# %timeit np.concatenate((total_tokens[pos - epsilon: pos +epsilon], context), axis=0)
    for pos in np.nditer(inserted_pos):
            context.append(total_tokens[pos - epsilon: pos])
            context.append( total_tokens[pos+1: pos + epsilon+1])
    context= np.concatenate((context), axis=0)
    uniq_context = np.unique(context)
    contexts.append(uniq_context.reshape(uniq_context.shape[0],1))
    subjects.append(inserted_tokens)

In [230]:
# %timeit set(np.unique(context))
# %timeit set(context.flatten())
unique_contexts = np.unique(np.concatenate( contexts, axis=0 ))
context_vec_list = [np.sum(( unique_contexts == con), axis = 0, keepdims= True) for con in contexts]
# context_vectors = np.zeros((len(revisions), uniq_context.shape[0]))
context_vectors = np.concatenate((context_vec_list), axis=0)
# X = np.empty((len(revisions), unique_contexts.shape[0]))

# contexts_vec_1 = np.array(np.sum(( unique_contexts == contexts[0]), axis = 0, keepdims= True))
# contexts_vec_2 = np.array(np.sum(( unique_contexts == contexts[1]), axis = 0, keepdims= True))
# contexts_vec_3 = np.array(np.sum(( unique_contexts == contexts[2]), axis = 0, keepdims= True))

# points = np.concatenate((contexts_vec_1, contexts_vec_2, contexts_vec_3), axis=0)
# context_vectors.shape

In [241]:
context_vectors.shape

(43, 2434)

#### Starting clustering below

In [ ]:
from sklearn.cluster import DBSCAN
from sklearn.metrics import pairwise

In [248]:
precomputd_X = pairwise.pairwise_distances(context_vectors, metric = "jaccard")
np.unique(precomputd_X)

/Users/aadilrasheed/anaconda/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


array([ 0.        ,  0.64705882,  0.70338983,  0.73333333,  0.7352415 ,
        0.75      ,  0.75690608,  0.78828829,  0.79144385,  0.79166667,
        0.80769231,  0.84615385,  0.84848485,  0.85507246,  0.86786297,
        0.86842105,  0.87434555,  0.8757764 ,  0.8875502 ,  0.88926746,
        0.88957055,  0.89      ,  0.89411765,  0.89423077,  0.8968254 ,
        0.90322581,  0.90625   ,  0.90697674,  0.90825688,  0.91020408,
        0.91647332,  0.91735537,  0.91810345,  0.92079208,  0.92134831,
        0.92364532,  0.92789374,  0.93103448,  0.93186373,  0.93243243,
        0.93589744,  0.93661972,  0.9382716 ,  0.94327731,  0.94392523,
        0.94402985,  0.94565217,  0.94736842,  0.94751773,  0.94779116,
        0.95090909,  0.95121951,  0.95169082,  0.95283019,  0.95391705,
        0.95454545,  0.95555556,  0.95750708,  0.95833333,  0.95893224,
        0.95934959,  0.96      ,  0.96022727,  0.96028881,  0.96111111,
        0.96160878,  0.96222664,  0.96226415,  0.96282528,  0.96

In [253]:
db = DBSCAN(eps=0.01, min_samples = 2 ).fit(precomputd_X)
db.labels_

array([-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  0, -1, -1, -1,
       -1, -1,  0, -1, -1, -1, -1, -1, -1])

In [ ]:
unique_clusters = set(db.labels_) - {-1}


In [168]:
ls -l

total 447904
-rw-r--r--  1 aadilrasheed  staff  222189264 Jan  2 20:48 Mama's_Family.h5
-rw-r--r--  1 aadilrasheed  staff         15 Jan  2 07:49 README.md
-rw-r--r--  1 aadilrasheed  staff    1815104 Dec 29 15:49 mama_700034288.h5
-rw-r--r--  1 aadilrasheed  staff    3522480 Dec 26 21:32 test.h5
-rw-r--r--  1 aadilrasheed  staff    1667191 Dec 26 21:35 wikiwho.ipynb
-rw-r--r--  1 aadilrasheed  staff      14137 Jan  2 20:48 wikiwho_context_clustering.ipynb
-rw-r--r--  1 aadilrasheed  staff     100950 Jan  2 20:48 wikiwho_neighbour.ipynb


In [172]:
rm Mama*